<center><i>Name: Pham Vinh Khang</i></center>

<center><i>Student's ID: 19126007</i></center>

# Define the data
The data includes the following:

- `weights`: A vector containing the weights of the items.

- `values`: A vector containing the values of the items.

- `capacity`: The capacity of the knapsack.

In [1]:
import numpy as np

values = np.random.randint(low=10, high=1000, size=10)
weights = np.random.randint(low=1, high=25, size=10)
capacity = np.random.randint(25, 50) + np.min(weights)

In [2]:
values

array([486, 362, 990, 573,  50,  82, 419, 712, 774, 706])

In [3]:
weights

array([ 8,  4,  6, 22, 13,  3, 11, 15, 13, 23])

In [4]:
capacity

44

# Naive Greedy Approach
By picking the most valuable items at each pass, we can have a fairly good (albeit not optimal)
solution for the problem.

Time complexity: $O(n^2)$ with $n$ is the number of items.

In [5]:
def naive_greedy(values, weights, capacity):
    """Solve the knapsack problem by a naive greedy approach."""
    
    knapsack = []
    remaining = capacity
    arg_values = np.argsort(values)
    
    for ind in arg_values[::-1]:
        if remaining <= 0:
            break
        if remaining > weights[ind]:
            knapsack.append((values[ind], weights[ind]))
            remaining -= weights[ind]
    return knapsack

In [6]:
knapsack_1 = naive_greedy(values, weights, capacity)

print(f'Knapsack: {knapsack_1}')
print(f'Total value: {sum(value for value, _ in knapsack_1)}')
print(f'Total weight: {sum(weight for _, weight in knapsack_1)}')

Knapsack: [(990, 6), (774, 13), (712, 15), (486, 8)]
Total value: 2962
Total weight: 42


# Dynamic Programmming
Let's define the solution as following:

1. We build a matrix `dp` whose column represents 
    each number from **min item weight** to total **capacity**
    and each row stands for each item.
    
2. The value of each cell `dp[i, j]` indicates the max value 
    that fits in the sub-knapsack with capacity `j+1`
    while considering all items from 1 to `i`.
    
3. Rule to fill in the cell:

    | ![](images/knapsack-dp.png) |
    |:--:|
    | *Grokking Algorithms* (170), by Aditya. Bhargava, 2016. |

4. The max value possible is the value of the bottom-right cell.

In [7]:
def knapsack_dp(values, weights, capacity):
    """Solve the knapsack problem using dynamic programming."""
    
    # Initialize the dp matrix
    # shape: len(values)+1, capacity+1
    # since we need to align the rows with the cols
    dp = np.zeros((len(values)+1, capacity+1), dtype=np.int32)
    
    for i in range(dp.shape[0]):
        for j in range(dp.shape[1]):
            if i*j == 0:
                continue  # skip if on the margin.
            
            # prev_max: previous max
            # cur_val: current value
            # remaining: value of remaining space
            # candidate: candidate value to update knapsack
            prev_max = dp[i-1, j]
            cur_val = values[i-1]
            remaining = dp[i-1, j-weights[i-1]]
            candidate = cur_val + remaining
            
            # Only consider update if
            # the new item fit the knapsack
            # Otherwise just use the previous max
            if weights[i-1] <= j:
                dp[i, j] = max(candidate, prev_max)
            else:
                dp[i, j] = dp[i-1, j]
                
    return dp[-1, -1]

In [8]:
print(f'Result = {knapsack_dp(values, weights, capacity)}')

Result = 3031


**Verify the result by using a [library](https://developers.google.com/optimization) of Google.**

The solver from the library expect a slightly different input format.
So we will need to do some funny reshape.

In [9]:
capacity = [capacity]
weights = np.expand_dims(weights, axis=0).tolist()

In [10]:
from ortools.algorithms import pywrapknapsack_solver

In [11]:
solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.
    KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')

In [12]:
solver.Init(values, weights, capacity)
computed_value = solver.Solve()
packed_items = []
packed_weights = []
total_weight = 0
print('Total value =', computed_value)
for i in range(len(values)):
    if solver.BestSolutionContains(i):
        packed_items.append(i)
        packed_weights.append(weights[0][i])
        total_weight += weights[0][i]
print('Total weight:', total_weight)
print('Packed items:', packed_items)
print('Packed_weights:', packed_weights)

Total value = 3031
Total weight: 42
Packed items: [0, 1, 2, 6, 8]
Packed_weights: [8, 4, 6, 11, 13]
